In [1]:
import glob
import random
import pandas as pd
import numpy as np
import time
import multiprocessing
from multiprocessing import Pool
from functools import partial
from multiprocessing import Manager

In [2]:
shingle_size = 5
jaccard_threshold = 0.25
minhash_threshold = 0.25

In [3]:
def get_shingles(filename):
    print("==== Creating shingles for file: {}".format(filename))
    shingles_set = set()

    with open(filename, 'rt', encoding='latin1') as file:
        for line in file:
            line = line.strip().lower()
            shingles_set.update([line[start: (start + shingle_size)] for start in range(0, len(line) - shingle_size)])
    
    shingles_set = set(map(lambda x: x.replace(" ","_"), shingles_set))

    print("===== Done ======")
    #print(shingles_set)
    return shingles_set

In [4]:
data_dir = "./new_data"
filenames = [f for f in glob.glob(data_dir+"/*")]
documents_shingles = [get_shingles(f) for f in filenames]

print(len(documents_shingles))

==== Creating shingles for file: ./new_data/accuracy_garmin_nuvi_255W_gps.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/performance_netbook_1005ha.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/display_garmin_nuvi_255W_gps.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/screen_ipod_nano_8gb.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/performance_honda_accord_2008.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/battery-life_ipod_nano_8gb.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/quality_toyota_camry_2007.txt.data
===== Done ======
==== Creating shingles for file: ./new_data/speed_garmin_nuvi_255W_gps.txt.data
===== Done ======
8


In [5]:
def jaccard_similarity(set_one, set_two):
    intersection = set_one.intersection(set_two)
    union = set_one.union(set_two)
    return len(intersection)/len(union)

In [7]:
total_docs = len(filenames)
similar_docs = []

sim_scores = []

for i in range(total_docs):
    j = i+1
    while j < total_docs:
        
        #print("*** Computing similarity between file:{} and file:{} ***".format(filenames[i],filenames[j]))
        sim = jaccard_similarity(documents_shingles[i], documents_shingles[j])
        #print(sim)
        sim_scores.append(sim)
        
        if sim >= jaccard_threshold :
              similar_docs.append((filenames[i],filenames[j],jaccard_similarity))

        j += 1      
              
print("=========== Similar documents are ===========")
for similar_doc in similar_docs:
    print("{} and {}".format(similar_doc[0], similar_doc[1]))
print("=============================================")

sim_scores = sorted(sim_scores)
print("Range of similarity scores is {} to {}".format(sim_scores[0],sim_scores[-1]))

=========== Similar documents are ===========
./new_data/display_garmin_nuvi_255W_gps.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
Range of similarity scores is 0.09442776109442776 to 0.26670146137787054


In [8]:
#Generating all the shingles space

all_shingles_space = set()
for doc_shingle in documents_shingles:
    all_shingles_space.update(doc_shingle)
    
all_shingles_space = list(all_shingles_space)
print(len(all_shingles_space))

15188


In [9]:

documents_vec = [[] for i in range(total_docs)]
for shingle in all_shingles_space:
    for i in range(total_docs):
        documents_vec[i].append(int(shingle in documents_shingles[i]))
        
print("=========Total rows ===== : {}".format(len(documents_vec[0])))
#print(documents_vec[0])
print("===============================")

=========Total rows ===== : 15188


In [10]:
df_data = {}
for doc_id,doc_vec in enumerate(documents_vec):
    df_data["Doc"+str(doc_id+1)] = doc_vec
    
vec_df = pd.DataFrame(df_data)
print(len(vec_df))
vec_df.head()

15188


,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8
0,0,1,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0


In [11]:
num_rows = vec_df.shape[0]

# hashing functions of the form : (a(x)+b)mod(num_columns)

num_hash_fn = 100
all_a = [random.randint(num_rows+10,num_rows+1000) for i in range(num_hash_fn)]
all_b = [random.randint(num_rows+2000,num_rows+3000) for i in range(num_hash_fn)]

hash_values = list(zip(all_a, all_b))
num_columns = total_docs

In [12]:
def hash_row(a,b,r,n):
    return int((a*r + b)%n)

In [13]:
sig_data = np.full((num_hash_fn,total_docs),np.inf)
sig_cols = ["Doc"+str(i) for i in range(1,total_docs+1)]

signature_df = pd.DataFrame(data=sig_data, columns=sig_cols)
signature_df.head()

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8
0,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf


In [13]:
print(num_rows)

15188


In [14]:
# Implementing min hash algorithm (TODO: Optimize this step, using multi-cores)
start_time = time.time() 

for row in range(num_rows):
    # Computing all hash functions for a row
    hash_outputs = []
    for i in range(num_hash_fn):
        hash_outputs.append(hash_row(*hash_values[i], row, num_rows))
        
    cols_values = vec_df.loc[row]
    
    for col_id, col_val in enumerate(cols_values):
        if col_val == 1:
            for hash_id, hash_val in enumerate(hash_outputs):
                if hash_val < signature_df.loc[hash_id][col_id]:
                    signature_df.loc[hash_id][col_id] = hash_val
                    
end_time = time.time() # for profiling

print("took {0} seconds.".format((end_time-start_time)))


took 283.4990425109863 seconds.


In [15]:
signature_df.head()

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8
0,5.0,6.0,11.0,3.0,2.0,6.0,6.0,0.0
1,3.0,3.0,3.0,3.0,7.0,3.0,3.0,3.0
2,1.0,3.0,5.0,1.0,3.0,3.0,5.0,3.0
3,0.0,2.0,8.0,10.0,4.0,10.0,4.0,0.0
4,1.0,5.0,3.0,1.0,3.0,3.0,1.0,3.0


In [17]:
# Finding similar items using min hashing
minhash_similar = []
for i in range(1,total_docs+1):
    j = i+1
    
    while j <= total_docs:
        #print("=== Computing similarity between file:{} and file:{} ===".format(filenames[i-1],filenames[j-1]))
        sim = np.sum(signature_df["Doc"+str(i)] == signature_df["Doc"+str(j)])/num_hash_fn
        #print(sim)

        if sim >= minhash_threshold :
              minhash_similar.append((filenames[i-1],filenames[j-1],sim))

        j += 1         
        
print("====== Similar docs using MinHashing are ============")
for similar_doc in minhash_similar:
    print("{} and {}".format(similar_doc[0], similar_doc[1]))
print("======================================================")

====== Similar docs using MinHashing are ============
./new_data/accuracy_garmin_nuvi_255W_gps.txt.data and ./new_data/performance_netbook_1005ha.txt.data
./new_data/accuracy_garmin_nuvi_255W_gps.txt.data and ./new_data/battery-life_ipod_nano_8gb.txt.data
./new_data/accuracy_garmin_nuvi_255W_gps.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
./new_data/performance_netbook_1005ha.txt.data and ./new_data/performance_honda_accord_2008.txt.data
./new_data/performance_netbook_1005ha.txt.data and ./new_data/battery-life_ipod_nano_8gb.txt.data
./new_data/display_garmin_nuvi_255W_gps.txt.data and ./new_data/performance_honda_accord_2008.txt.data
./new_data/display_garmin_nuvi_255W_gps.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
./new_data/screen_ipod_nano_8gb.txt.data and ./new_data/battery-life_ipod_nano_8gb.txt.data
./new_data/screen_ipod_nano_8gb.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
./new_data/battery-life_ipod_nano_8gb.txt.data and ./new_

In [18]:
# Defining parameters for lsh

b = 34
r = 3

t = pow(1/b, 1/r)
print("**************** Threshold chosen is {} ***************".format(t))

**************** Threshold chosen is 0.30867895949930413 ***************


In [19]:
# hashing functions of the form : (a(x)+b)mod(num_buckets)

num_buckets = 100003
num_lsh_hashes = b
all_a = [random.randint(0,num_buckets+100000) for i in range(num_lsh_hashes)]
all_b = [random.randint(0,num_buckets+100000) for i in range(num_lsh_hashes)]


lsh_hash_values = list(zip(all_a, all_b))

In [20]:
try:
    del lsh_hashes
except:
    pass


lsh_hashes = []

for i in range(b):
    lsh_hashes.append({})
    for j in range(total_docs):
        col_band = signature_df.loc[i*r:(i+1)*r-1, "Doc"+str(j+1)]
        col_int = int("".join([str(int(x)) for x in list(col_band)]))
        
        hash_output = hash_row(*lsh_hash_values[i], col_int, num_buckets)
        
        if hash_output in lsh_hashes[i]:
            lsh_hashes[i][hash_output].update(["Doc"+str(j+1)])
        else:
            lsh_hashes[i][hash_output] = set(["Doc"+str(j+1)])

candidate_pairs = []
#print(lsh_hashes)
for lsh_hash in lsh_hashes:
    for k,v in lsh_hash.items():
        if len(v) > 1:
            candidate_pairs.append(tuple(sorted(list(v))))

candidate_pairs = set(candidate_pairs)            
print("====== Candidate pairs are =======")
[print(x) for x in candidate_pairs]
print("==================================")

====== Candidate pairs are =======
('Doc1', 'Doc8')
('Doc2', 'Doc8')
('Doc3', 'Doc8')
('Doc1', 'Doc7')
('Doc4', 'Doc6', 'Doc8')
('Doc5', 'Doc8')
('Doc1', 'Doc2')
('Doc3', 'Doc7')
('Doc4', 'Doc5')
('Doc2', 'Doc6')


In [22]:
lsh_similar = []
for candidate_pair in candidate_pairs:
        file1 = int(candidate_pair[0][3:])-1
        file2 = int(candidate_pair[1][3:])-1
        #print("=== Computing similarity between file:{} and file:{} ===".format(filenames[file1],filenames[file2]))
        sim = np.sum(signature_df[candidate_pair[0]] == signature_df[candidate_pair[1]])/num_hash_fn
        #print(sim)

        if sim >= minhash_threshold :
              lsh_similar.append((filenames[file1],filenames[file2],sim))

        j += 1         
        
print("====== Similar docs using LSH are ====================")
for similar_doc in lsh_similar:
    print("{} and {}".format(similar_doc[0], similar_doc[1]))
print("======================================================")

====== Similar docs using LSH are ====================
./new_data/accuracy_garmin_nuvi_255W_gps.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
./new_data/display_garmin_nuvi_255W_gps.txt.data and ./new_data/speed_garmin_nuvi_255W_gps.txt.data
./new_data/screen_ipod_nano_8gb.txt.data and ./new_data/battery-life_ipod_nano_8gb.txt.data
./new_data/accuracy_garmin_nuvi_255W_gps.txt.data and ./new_data/performance_netbook_1005ha.txt.data
./new_data/performance_netbook_1005ha.txt.data and ./new_data/battery-life_ipod_nano_8gb.txt.data
